In [2]:
import os
# Find the latest version of spark 3.2 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.2.3'
spark_version = 'spark-3.2.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:5 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:13 http://archive.ubuntu.co

In [3]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2023-01-04 00:10:15--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  5.47MB/s    in 0.2s    

2023-01-04 00:10:16 (5.47 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M17-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [5]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Home_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   33670092|R1UUISQ1GKOJTI|B00EE62UAE|     583436067|Trademark Home Po...|            Home|          1|            0|          0|   N|                Y|                 Run|Don't buy it clap...| 2015-08-31|
|         US|   13726692|R1HOJ9WE8VCVOD|B001APXO5C|     465035091|O2-Cool 10-Inch P...|            Home|          5|    

In [6]:
# Filter total votes >= 20
df_votes = df.filter("total_votes>=20")
df_votes.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   32555853|R27XX9MG7OWSSM|B00CMV99HU|     475264259|5 x 20 Quality Ca...|            Home|          5|           49|         51|   N|                Y|Made Curtains out...|I bought these fo...| 2015-08-31|
|         US|   24278614|R1I4IFDOW6IW9O|B00ZAUWZ6M|     195636450|Ellington Home 18...|            Home|          3|    

In [7]:
# Filter helpful votes / total votes >= 50%
from pyspark.sql import functions as F
df_helpful = df_votes.filter((F.col("helpful_votes")/F.col("total_votes"))>=0.5)
df_helpful.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   32555853|R27XX9MG7OWSSM|B00CMV99HU|     475264259|5 x 20 Quality Ca...|            Home|          5|           49|         51|   N|                Y|Made Curtains out...|I bought these fo...| 2015-08-31|
|         US|   24278614|R1I4IFDOW6IW9O|B00ZAUWZ6M|     195636450|Ellington Home 18...|            Home|          3|    

In [8]:
# Filter vine (paid) reviews
df_vine_y = df_helpful.filter(df_helpful['vine'] == 'Y')
df_vine_y.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   52815715|R2W1OMYJERBXJ1|B00SFUVNKS|     977603375|Panasonic MC-CL94...|            Home|          5|           20|         23|   Y|                N|Compact and powerful|With pets in the ...| 2015-08-31|
|         US|   13663596| R31VJCBBKU13Y|B0105TXVRE|     820889787|Electrolux EL4335...|            Home|          5|    

In [9]:
# Number of vine (paid) reviews
df_vine_y.agg({"review_id":"count"}).show()

+----------------+
|count(review_id)|
+----------------+
|            1448|
+----------------+



In [10]:
# Filter non-vine (unpaid) reviews
df_vine_n = df_helpful.filter(df_helpful['vine'] == 'N')
df_vine_n.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   32555853|R27XX9MG7OWSSM|B00CMV99HU|     475264259|5 x 20 Quality Ca...|            Home|          5|           49|         51|   N|                Y|Made Curtains out...|I bought these fo...| 2015-08-31|
|         US|   24278614|R1I4IFDOW6IW9O|B00ZAUWZ6M|     195636450|Ellington Home 18...|            Home|          3|    

In [11]:
# Number of non-vine (unpaid) reviews
df_vine_n.agg({"review_id":"count"}).show()

+----------------+
|count(review_id)|
+----------------+
|           90768|
+----------------+



In [12]:
# Total number of (helpful) reviews
df_reviews = df_helpful.agg({"review_id":"count"})
pd_reviews = df_reviews.toPandas()
pd_reviews.head()

,count(review_id)
0,92216


In [13]:
# Number of total 5-star reviews
df_star = df.filter(df["star_rating"] == 5).agg({"review_id":"count"})
df_star.show()

+----------------+
|count(review_id)|
+----------------+
|         3892997|
+----------------+



In [14]:
import pandas as pd

In [ ]:
# Percentage of Vine 5 star reviews
df_paid_five_star = df_vine_y.filter(df_vine_y['star_rating']==5)
pd_paid = df_paid_five_star.toPandas().count()
pd_paid_pct = pd_paid['review_id']/pd_reviews
pd_paid_pct.loc[:, "count(review_id)"] = pd_paid_pct["count(review_id)"].map('{:.2%}'.format)
pd_paid_pct.head()

,count(review_id)
0,0.70%


In [16]:
# Number of Vine reviews that are 5 stars
count_paid_five = df_paid_five_star.agg({'review_id':'count'})
count_paid_five.show()

+----------------+
|count(review_id)|
+----------------+
|             647|
+----------------+



In [17]:
# Percentage of non-Vine 5 star reviews
df_unpaid_five_star = df_vine_n.filter(df_vine_n['star_rating']==5)
pd_unpaid = df_unpaid_five_star.toPandas().count()
pd_unpaid_pct = pd_unpaid['review_id']/pd_reviews
pd_unpaid_pct.loc[:, "count(review_id)"] = pd_unpaid_pct["count(review_id)"].map('{:.2%}'.format)
pd_unpaid_pct.head()

,count(review_id)
0,47.83%


In [18]:
# Number of non-Vine reviews that are 5 stars
count_unpaid_five = df_unpaid_five_star.agg({'review_id':'count'})
count_unpaid_five.show()

+----------------+
|count(review_id)|
+----------------+
|           44104|
+----------------+

